In [699]:
import folium
import time
import math
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from ortools.linear_solver import pywraplp
from shapely.geometry import LineString, Point
from itertools import permutations
# pd.set_option('display.float_format', '{:.10f}'.format)

In [700]:
# import math

# # 시설 데이터 (예시)
# facilities = [
#     {"index": 103, "demand": 1289}, {"index": 1, "demand": 113}, {"index": 56, "demand": 715},
#     {"index": 71, "demand": 1083}, {"index": 2, "demand": 608}, {"index": 89, "demand": 1482},
#     {"index": 57, "demand": 1391}, {"index": 85, "demand": 1500}, {"index": 88, "demand": 1499},
#     {"index": 3, "demand": 1494}, {"index": 86, "demand": 954}, {"index": 0, "demand": 1494},
#     {"index": 11, "demand": 1486}, {"index": 41, "demand": 1494}, {"index": 45, "demand": 1236},
#     {"index": 82, "demand": 1488}, {"index": 100, "demand": 1479}
# ]

# # 전체 차량 개수와 예산
# total_vehicles = 70
# total_budget = 1000

# # 1. 총 수요량 계산
# total_demand = sum(f["demand"] for f in facilities)

# # 2. 비율 계산 및 할당
# for facility in facilities:
#     demand_ratio = facility["demand"] / total_demand
#     facility["vehicles"] = math.ceil(total_vehicles * demand_ratio)  # 차량 수 (반올림)
#     facility["budget"] = round(total_budget * demand_ratio)         # 예산 할당

# # 결과 출력
# for facility in facilities:
#     print(f"시설 인덱스: {facility['index']}, 커버 수요량: {facility['demand']}, "
#           f"할당 차량: {facility['vehicles']}, 할당 예산: {facility['budget']}")

In [701]:
shp_file_path_1 = "Data\남구 65세 이상 기초수급자 거주지(통신사비율보정) - 복사본\남구 65세이상 기초수급자(통산사비율보정).shp"
df_Namgu = gpd.read_file(shp_file_path_1, encoding='cp949')

우선순위 점수에 정규화 적용

In [702]:
# # 가중치 설정 --> 나중에 민감도 분석
# weight_demand = 0.5
# weight_price = 0.5

# # 수요 및 집값의 최소값, 최대값 계산
# demand_min = df_Namgu['demand'].min()
# demand_max = df_Namgu['demand'].max()
# price_min = df_Namgu['aver_price'].min()
# price_max = df_Namgu['aver_price'].max()

# # 수요값 정규화
# df_Namgu['demand_norm'] = (df_Namgu['demand'] - demand_min) / (demand_max - demand_min)

# # 집값 정규화 (집값이 낮을수록 높은 점수) 
# df_Namgu['price_norm'] = (df_Namgu['aver_price'] - price_min) / (price_max - price_min)
# df_Namgu['price_score'] = 1 / (df_Namgu['price_norm'] + 1e-6)  # 작은 epsilon 값 추가

# # 최종 우선순위 점수 계산
# df_Namgu['priority_score'] = weight_demand * df_Namgu['demand_norm'] + weight_price * df_Namgu['price_score']

# # 결과 출력
# print(df_Namgu.head())
# print(df_Namgu.shape)

In [703]:
# 가중치 설정 --> 나중에 민감도 분석
weight_demand = 0.5
weight_price = 0.5

# 수요 및 집값의 최소값, 최대값 계산
demand_min = df_Namgu['demand'].min()
demand_max = df_Namgu['demand'].max()
price_min = df_Namgu['aver_price'].min()
price_max = df_Namgu['aver_price'].max()

# 수요값 정규화
df_Namgu['demand_norm'] = (df_Namgu['demand'] - demand_min) / (demand_max - demand_min)

# 집값 log 변환 후 정규화
df_Namgu['price_log'] = np.log(df_Namgu['aver_price'] + 1)  # 1 더해줘서 0인 경우 방지
price_log_min = df_Namgu['price_log'].min()
price_log_max = df_Namgu['price_log'].max()
df_Namgu['price_norm'] = (df_Namgu['price_log'] - price_log_min) / (price_log_max - price_log_min)

# 최종 우선순위 점수 계산
df_Namgu['priority_score'] = weight_demand * df_Namgu['demand_norm'] + weight_price * df_Namgu['price_norm']

# 결과 출력
print(df_Namgu.head())
print(df_Namgu.shape)

   aver_price  demand                    geometry  demand_norm  price_log  \
0   603211.69      15  POINT (129.31711 35.52537)         0.10      13.31   
1   771516.20      12  POINT (129.32104 35.52112)         0.06      13.56   
2   808496.62      10  POINT (129.31717 35.52442)         0.04      13.60   
3   914786.27      13   POINT (129.3178 35.52161)         0.08      13.73   
4  1009717.94       9  POINT (129.31596 35.52304)         0.03      13.83   

   price_norm  priority_score  
0        0.25            0.18  
1        0.41            0.24  
2        0.43            0.24  
3        0.51            0.29  
4        0.57            0.30  
(1379, 7)


In [704]:
# # 가중치 설정 --> 나중에 민감도 분석
# weight_demand = 0.5
# weight_price = 0.5

# # 집값 기반 점수 계산 (집값이 낮을수록 높은 점수)
# df_Namgu['price_score'] = 1 / df_Namgu['aver_price']

# # 최종 우선순위 점수 계산
# df_Namgu['priority_score'] = weight_demand * df_Namgu['demand'] + weight_price * df_Namgu['price_score']

# # 결과 출력
# print(df_Namgu.head())
# print(df_Namgu.shape)

In [705]:
shp_file_path_2 = "Data/result_shp/all_facility_demand_coverage.shp"
df_Coverage = gpd.read_file(shp_file_path_2, encoding='cp949')
df_Coverage

,facility_i,facility_g,geometry
0,F1,POINT (129.3299 35.5299),POINT (129.32903 35.52958)
1,F1,POINT (129.3299 35.5299),POINT (129.32376 35.52522)
2,F1,POINT (129.3299 35.5299),POINT (129.32946 35.52315)
3,F1,POINT (129.3299 35.5299),POINT (129.32187 35.52672)
4,F1,POINT (129.3299 35.5299),POINT (129.32372 35.52574)
...,...,...,...
1374,F17,POINT (129.2594203073972 35.54925746853039),POINT (129.2683 35.55187)
1375,F17,POINT (129.2594203073972 35.54925746853039),POINT (129.26388 35.54963)
1376,F17,POINT (129.2594203073972 35.54925746853039),POINT (129.26882 35.55172)
1377,F17,POINT (129.2594203073972 35.54925746853039),POINT (129.26436 35.55147)


In [706]:
final_gdf = gpd.GeoDataFrame(df_Namgu.merge(df_Coverage, on='geometry', how='inner'))
final_gdf

,aver_price,demand,geometry,demand_norm,price_log,price_norm,priority_score,facility_i,facility_g
0,603211.69,15,POINT (129.31711 35.52537),0.10,13.31,0.25,0.18,F12,POINT (129.31452557761335 35.5273436555103)
1,771516.20,12,POINT (129.32104 35.52112),0.06,13.56,0.41,0.24,F3,POINT (129.3187 35.514)
2,808496.62,10,POINT (129.31717 35.52442),0.04,13.60,0.43,0.24,F7,POINT (129.32260612143162 35.53247598621677)
3,914786.27,13,POINT (129.3178 35.52161),0.08,13.73,0.51,0.29,F3,POINT (129.3187 35.514)
4,1009717.94,9,POINT (129.31596 35.52304),0.03,13.83,0.57,0.30,F3,POINT (129.3187 35.514)
...,...,...,...,...,...,...,...,...,...
1374,1415100.00,7,POINT (129.30676 35.54425),0.00,14.16,0.78,0.39,F9,POINT (129.31239017292043 35.539695438459965)
1375,1418795.91,22,POINT (129.3078 35.54577),0.19,14.17,0.78,0.49,F9,POINT (129.31239017292043 35.539695438459965)
1376,1488694.87,24,POINT (129.30743 35.54561),0.22,14.21,0.81,0.52,F9,POINT (129.31239017292043 35.539695438459965)
1377,1555998.00,18,POINT (129.30681 35.54225),0.14,14.26,0.84,0.49,F9,POINT (129.31239017292043 35.539695438459965)


In [707]:
# facility_id 결정
final_gdf = final_gdf[final_gdf['facility_i'] == 'F14']
final_gdf

,aver_price,demand,geometry,demand_norm,price_log,price_norm,priority_score,facility_i,facility_g
25,1724398.00,46,POINT (129.32003 35.52788),0.51,14.36,0.90,0.70,F14,POINT (129.32091474461058 35.53110010746248)
63,902086.07,49,POINT (129.32174 35.52698),0.55,13.71,0.50,0.52,F14,POINT (129.32091474461058 35.53110010746248)
84,1041825.10,17,POINT (129.32269 35.52692),0.13,13.86,0.59,0.36,F14,POINT (129.32091474461058 35.53110010746248)
117,1191726.95,19,POINT (129.32281 35.52953),0.16,13.99,0.67,0.41,F14,POINT (129.32091474461058 35.53110010746248)
126,1262175.00,18,POINT (129.32104 35.52654),0.14,14.05,0.71,0.43,F14,POINT (129.32091474461058 35.53110010746248)
...,...,...,...,...,...,...,...,...,...
1286,994916.49,10,POINT (129.31238 35.53387),0.04,13.81,0.56,0.30,F14,POINT (129.32091474461058 35.53110010746248)
1291,1022504.89,7,POINT (129.31225 35.53574),0.00,13.84,0.58,0.29,F14,POINT (129.32091474461058 35.53110010746248)
1300,1073000.73,48,POINT (129.31315 35.53498),0.53,13.89,0.61,0.57,F14,POINT (129.32091474461058 35.53110010746248)
1303,1115331.17,23,POINT (129.3138 35.53389),0.21,13.92,0.63,0.42,F14,POINT (129.32091474461058 35.53110010746248)


In [708]:
max_value = final_gdf['priority_score'].max()
min_value = final_gdf['priority_score'].min()

print("최대값:", max_value)
print("최소값:", min_value)

최대값: 0.724900713992041
최소값: 0.08826241021026918


노드 데이터 shp 파일 저장

In [709]:
final_gdf = final_gdf.rename(columns={
    'aver_price': 'avg_price',
    'price_log': 'price_log',
    'price_norm': 'price_nrm',
    'priority_score': 'Prty score',
    'facility_i': 'fac_i',
    'facility_g': 'fac_g'
})

final_gdf = final_gdf.set_crs("EPSG:4326")

In [710]:
output_path = r"C:\Users\jymin\OneDrive\OR LAB\(2024) Orienteering Problem\Code\Data/result_shp/final_gdf_14_v2.shp"

# GeoDataFrame을 Shapefile로 저장
final_gdf.to_file(output_path, driver='ESRI Shapefile')

C:\Users\jymin\AppData\Local\Temp\ipykernel_5324\707126392.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  final_gdf.to_file(output_path, driver='ESRI Shapefile')
c:\Users\jymin\AppData\Local\Programs\Python\Python39\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'demand_norm' to 'demand_nor'
  ogr_write(


In [639]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# # 히스토그램과 KDE 그래프 그리기
# plt.figure(figsize=(10, 6))
# sns.histplot(final_gdf['priority_score'], kde=True, bins=30, color='blue', edgecolor='black')
# plt.title('Priority Score Distribution', fontsize=16)
# plt.xlabel('Priority Score', fontsize=14)
# plt.ylabel('Frequency', fontsize=14)
# plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.show()

필터링

In [640]:
# def split_demand_rows(df, max_demand=9):
#     split_rows = []
#     for _, row in df.iterrows():
#         demand = row["demand"]
#         while demand > max_demand:
#             # 최대 max_demand 만큼 분할된 행 추가
#             split_row = row.copy()
#             split_row["demand"] = max_demand
#             split_rows.append(split_row)
#             demand -= max_demand
#         # 남은 수요가 있을 때 행 추가
#         if demand > 0:
#             split_row = row.copy()
#             split_row["demand"] = demand
#             split_rows.append(split_row)
    
#     # 새로운 데이터프레임 생성 후 인덱스 재설정
#     split_df = pd.DataFrame(split_rows)
#     split_df.reset_index(drop=True, inplace=True)
#     return split_df

# final_gdf = split_demand_rows(final_gdf)

In [641]:
# def select_coordinate(df):
#     # Point를 문자열 좌표로 변환 (또는 tuple 사용 가능)
#     df['coordinate_str'] = df['geometry'].apply(lambda p: (p.x, p.y))

#     # 좌표별로 최대 3개씩 선택
#     selected = df.groupby('coordinate_str').head(3)

#     # 변환 열 제거 (필요 없는 경우)
#     selected = selected.drop(columns=['coordinate_str'])

#     # 인덱스 재설정
#     selected.reset_index(drop=True, inplace=True)
#     return selected

# filtered_final_gdf = select_coordinate(final_gdf)

# print(filtered_final_gdf)

In [642]:
# # priority_score의 중앙값 계산
# median_priority_score = final_gdf['priority_score'].median()

# # priority_score가 중앙값 이상인 값만 필터링
# filtered_final_gdf = final_gdf[final_gdf['priority_score'] >= median_priority_score]

In [643]:
# print(filtered_final_gdf)
# print(len(filtered_final_gdf))

In [644]:
filtered_final_gdf = final_gdf.sort_values(by='priority_score', ascending=False).head(15)
# filtered_final_gdf = filtered_final_gdf.sort_values(by='priority_score', ascending=False).head(30)

In [645]:
total_demand = filtered_final_gdf['demand'].sum()
total_priority_score = filtered_final_gdf['priority_score'].sum()

print(f"Total Demand: {total_demand}")
print(f"Total Priority Score: {total_priority_score}")

Total Demand: 308
Total Priority Score: 7.229270844503699


최종

In [652]:
import pandas as pd
import matplotlib.pyplot as plt
from ortools.linear_solver import pywraplp
import math
from shapely.geometry import LineString
import folium

# Haversine 공식을 사용한 거리 계산 함수 (킬로미터 단위)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # 지구 반지름 (킬로미터)
    lat1_rad, lon1_rad = math.radians(lat1), math.radians(lon1)
    lat2_rad, lon2_rad = math.radians(lat2), math.radians(lon2)
    dlat, dlon = lat2_rad - lat1_rad, lon2_rad - lon1_rad
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))

def calculate_distances(coords):
    num_nodes = len(coords)
    distances = [[0] * num_nodes for _ in range(num_nodes)]
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                lat1, lon1 = coords[i]
                lat2, lon2 = coords[j]
                distances[i][j] = haversine(lat1, lon1, lat2, lon2)
    return distances

def solve_ctop_with_budget_constraint(num_nodes, num_vehicles, distances, scores, demands, vehicle_capacity, travel_cost_per_unit, budget):
    solver = pywraplp.Solver.CreateSolver('CBC')

    # 변수 정의
    x = {}
    y = {}
    for i in range(num_nodes):
        for j in range(num_nodes):
            for k in range(num_vehicles):
                x[i, j, k] = solver.BoolVar(f'x_{i}_{j}_{k}')
        for k in range(num_vehicles):
            y[i, k] = solver.BoolVar(f'y_{i}_{k}')

    # 목적 함수 정의 (총 점수 최대화)
    solver.Maximize(
        solver.Sum(scores[i] * y[i, k] for i in range(1, num_nodes) for k in range(num_vehicles))
    )

    # 제약 조건 정의
    # 1. 각 노드는 하나의 차량에만 할당될 수 있음
    for i in range(1, num_nodes):
        solver.Add(solver.Sum(y[i, k] for k in range(num_vehicles)) <= 1)

    # 2. 차량별 제약 조건
    for k in range(num_vehicles):
        # 차량 용량 제약
        solver.Add(solver.Sum(demands[i] * y[i, k] for i in range(1, num_nodes)) <= vehicle_capacity[k])

        # 차량의 경로 흐름 제약
        for i in range(num_nodes):
            solver.Add(solver.Sum(x[i, j, k] for j in range(num_nodes) if j != i) == y[i, k])
            solver.Add(solver.Sum(x[j, i, k] for j in range(num_nodes) if j != i) == y[i, k])

        # 차량은 Depot(출발지)에서 출발하고 돌아와야 함
        solver.Add(solver.Sum(x[0, j, k] for j in range(1, num_nodes)) == 1)
        solver.Add(solver.Sum(x[i, 0, k] for i in range(1, num_nodes)) == 1)

    # 총 이동 비용 제약 (강화된 예산 제약)
    solver.Add(
    solver.Sum(
        distances[i][j] * travel_cost_per_unit * x[i, j, k]
        for i in range(num_nodes) for j in range(num_nodes) for k in range(num_vehicles)
    ) <= budget)

    # 문제 풀이
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        total_score, total_cost, total_demand_covered, total_distance = 0, 0, 0, 0
        routes = []
        for k in range(num_vehicles):
            route = [0]
            for i in range(1, num_nodes):
                if y[i, k].solution_value() > 0.5:
                    route.append(i)
            route.append(0)
            routes.append(route)

            travel_cost = sum(distances[route[i]][route[i + 1]] * travel_cost_per_unit for i in range(len(route) - 1))
            total_distance += sum(distances[route[i]][route[i + 1]] for i in range(len(route) - 1))
            total_cost += travel_cost
            total_demand_covered += sum(demands[i] for i in route if i != 0)
            total_score += sum(scores[i] for i in route if i != 0)

        return total_score, total_cost, total_demand_covered, total_distance, routes
    return None, None, None, None, None

def visualize_routes_on_map(coords, routes, num_vehicles):
    # 중앙 위치를 울산으로 설정
    m = folium.Map(location=[35.5299, 129.3299], zoom_start=13)

    # 차량 경로 색상을 설정
    colors = ['red', 'green', 'blue', 'purple', 'orange', 'cyan', 'magenta', 'black', 'white', 'gray']
    
    # 경로를 지도에 추가
    for k, route in enumerate(routes):
        color = colors[k % len(colors)]
        
        # 경로의 각 지점을 PolyLine으로 연결하여 지도에 추가
        folium.PolyLine(
            locations=[(coords[node][1], coords[node][0]) for node in route],
            color=color,
            weight=5,
            opacity=0.7,
            popup=f'Vehicle {k + 1} Route'
        ).add_to(m)
        
        # 각 노드에 마커 추가
        for node in route:
            folium.Marker(
                location=[coords[node][1], coords[node][0]],
                popup=f'Node {node}',
                icon=folium.Icon(color='blue' if node == 0 else 'green')
            ).add_to(m)
    
    # 지도를 출력
    return m

# GeoDataFrame 데이터 입력
from shapely.wkt import loads  # WKT 문자열을 Geometry 객체로 변환

# depot_point를 final_gdf의 facility_g 열의 첫 번째 데이터로 설정
depot_point = loads(filtered_final_gdf.iloc[0]['facility_g'])  # 문자열을 Point 객체로 변환
depot_coord = (depot_point.x, depot_point.y)  # Point 객체에서 x, y 좌표 추출

node_coords = [(point.x, point.y) for point in filtered_final_gdf.geometry]
coords = [depot_coord] + node_coords

node_coords = [(point.x, point.y) for point in filtered_final_gdf.geometry]
coords = [depot_coord] + node_coords

scores = [0] + filtered_final_gdf['priority_score'].tolist()
demands = [0] + filtered_final_gdf['demand'].tolist()

distances = calculate_distances(coords)
num_nodes = len(coords)
num_vehicles = 3
vehicle_capacity = [45] * num_vehicles
travel_cost_per_unit = 6
budget = 20

# 문제 풀이
total_score, total_cost, total_demand_covered, total_distance, routes = solve_ctop_with_budget_constraint(
    num_nodes, num_vehicles, distances, scores, demands, vehicle_capacity, travel_cost_per_unit, budget
)

# 경로 시각화
if routes:

    # 전체 수요량 계산
    total_demand = final_gdf['demand'].sum()

    # 테이블 데이터 생성
    route_data = []
    for k, route in enumerate(routes):
        route_distance = sum(distances[route[i]][route[i + 1]] for i in range(len(route) - 1))
        route_cost = sum(distances[route[i]][route[i + 1]] * travel_cost_per_unit for i in range(len(route) - 1))
        route_score = sum(scores[node] for node in route if node != 0)
        route_demand_covered = sum(demands[node] for node in route if node != 0)
        
        # 비율 계산
        coverage_percentage = (route_demand_covered / total_demand) * 100

        route_data.append({
        "Vehicle": f"Vehicle {k + 1}",
        "Route": " -> ".join(map(str, route)),
        "Distance (km)": round(route_distance, 2),
        "Cost ($)": round(route_cost, 2),
        "Score": round(route_score, 2),
        "Demand Covered": route_demand_covered,
        "Coverage (%)": round(coverage_percentage, 2)  # 비율 추가
        })

    # 성과 요약 추가
    total_summary = {
    "Vehicle": "Total",
    "Route": "-",
    "Distance (km)": round(total_distance, 2),
    "Cost ($)": round(total_cost, 2),
    "Score": round(total_score, 2),
    "Demand Covered": total_demand_covered,
    "Coverage (%)": round((total_demand_covered / total_demand) * 100, 2)  # 전체 비율 계산
    }
    route_data.append(total_summary)

    # 데이터프레임으로 변환 후 출력
    df = pd.DataFrame(route_data)

    # final_gdf['facility_i']의 첫 번째 값을 모든 행에 넣기
    facility_value = filtered_final_gdf['facility_i'].iloc[0]
    df.insert(0, 'Facility', [facility_value] * len(df))

    # pandas의 출력 옵션 설정: 소수점 자릿수 제한
    pd.set_option('display.float_format', '{:.2f}'.format)

    display(df.style.hide(axis="index"))

    m = visualize_routes_on_map(coords, routes, num_vehicles)
    display(m)

else:
    print("No solution found.")

Facility,Vehicle,Route,Distance (km),Cost ($),Score,Demand Covered,Coverage (%)
F3,Vehicle 1,0 -> 7 -> 9 -> 10 -> 14 -> 0,1.670000,9.990000,1.810000,42,6.910000
F3,Vehicle 2,0 -> 4 -> 5 -> 12 -> 0,1.700000,10.210000,1.420000,41,6.740000
F3,Vehicle 3,0 -> 8 -> 13 -> 15 -> 0,1.530000,9.200000,1.290000,44,7.240000
F3,Total,-,4.900000,29.400000,4.520000,127,20.890000


shp 저장

In [653]:
import geopandas as gpd
from shapely.geometry import LineString, Point

def save_routes_and_nodes_to_shp(coords, routes, filtered_final_gdf, output_line_file, output_point_file):
    # LineString 데이터 준비
    route_geometries = []
    route_data = []

    # Point 데이터 준비
    node_geometries = []
    node_data = []
    
    # 노드별 priority_score를 사전 형태로 저장
    node_priority = {idx: score for idx, score in zip(filtered_final_gdf.index, filtered_final_gdf['priority_score'])}
    
    for k, route in enumerate(routes):
        # 각 차량의 경로를 LineString으로 생성
        route_coords = [(coords[node][0], coords[node][1]) for node in route]  # (lon, lat) 형식
        line = LineString(route_coords)
        
        # LineString 데이터 추가
        route_geometries.append(line)
        route_data.append({
            "Vehicle": f"Vehicle {k + 1}",
            "Route": " -> ".join(map(str, route))
        })
        
        # Point 데이터 추가
        for node in route:
            lon, lat = coords[node][0], coords[node][1]
            node_geometries.append(Point(lon, lat))
            node_data.append({
                "Node": node,
                "Priority": node_priority.get(node, None),
                "Vehicle": f"Vehicle {k + 1}"
            })
    
    # LineString GeoDataFrame 생성
    gdf_routes = gpd.GeoDataFrame(route_data, geometry=route_geometries, crs="EPSG:4326")
    gdf_routes.to_file(output_line_file, driver="ESRI Shapefile")
    
    # Point GeoDataFrame 생성
    gdf_nodes = gpd.GeoDataFrame(node_data, geometry=node_geometries, crs="EPSG:4326")
    gdf_nodes.to_file(output_point_file, driver="ESRI Shapefile")
    
    print(f"Routes Shapefile saved to {output_line_file}")
    print(f"Nodes Shapefile saved to {output_point_file}")

# Shapefile 저장 경로
output_line_shp = r"C:\Users\jymin\OneDrive\OR LAB\(2024) Orienteering Problem\Code\Data/result_shp/final_route_3.shp"
output_point_shp = r"C:\Users\jymin\OneDrive\OR LAB\(2024) Orienteering Problem\Code\Data/result_shp/final_point_3.shp"

# 경로 및 노드 데이터를 저장
save_routes_and_nodes_to_shp(coords, routes, filtered_final_gdf, output_line_shp, output_point_shp)

Routes Shapefile saved to C:\Users\jymin\OneDrive\OR LAB\(2024) Orienteering Problem\Code\Data/result_shp/final_route_3.shp
Nodes Shapefile saved to C:\Users\jymin\OneDrive\OR LAB\(2024) Orienteering Problem\Code\Data/result_shp/final_point_3.shp


In [648]:
# import geopandas as gpd
# from shapely.geometry import LineString

# # 차량별 경로를 GeoDataFrame으로 변환, output_file
# def save_routes_to_shp(coords, routes, output_file):
#     route_geometries = []
#     route_data = []
    
#     for k, route in enumerate(routes):
#         # 각 차량의 경로를 LineString으로 생성
#         route_coords = [(coords[node][0], coords[node][1]) for node in route]  # (lon, lat) 형식
#         line = LineString(route_coords)
        
#         # 데이터 저장 (Vehicle ID 및 Geometry)
#         route_geometries.append(line)
#         route_data.append({
#             "Vehicle": f"Vehicle {k + 1}",
#             "Route": " -> ".join(map(str, route))
#         })
    
#     # GeoDataFrame 생성
#     gdf = gpd.GeoDataFrame(route_data, geometry=route_geometries, crs="EPSG:4326")
#     # print(gdf)
#     # Shapefile로 저장
#     gdf.to_file(output_file, driver="ESRI Shapefile")



# # Shapefile 저장 경로 지정
# output_path = r"C:\Users\jymin\OneDrive\OR LAB\(2024) Orienteering Problem\Code\Data/result_shp/final_route_14.shp"

# # 차량 경로 데이터를 저장, output_path
# save_routes_to_shp(coords, routes, output_path)
# print(f"Shapefile saved to {output_path}")

테스트

In [649]:
# import pandas as pd
# import matplotlib.pyplot as plt
# from ortools.linear_solver import pywraplp
# import math
# from shapely.geometry import LineString
# import folium
# from shapely.wkt import loads  # WKT 문자열을 Geometry 객체로 변환

# # Haversine 공식을 사용한 거리 계산 함수 (킬로미터 단위)
# def haversine(lat1, lon1, lat2, lon2):
#     R = 6371.0  # 지구 반지름 (킬로미터)
#     lat1_rad, lon1_rad = math.radians(lat1), math.radians(lon1)
#     lat2_rad, lon2_rad = math.radians(lat2), math.radians(lon2)
#     dlat, dlon = lat2_rad - lat1_rad, lon2_rad - lon1_rad
#     a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
#     return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))

# def calculate_distances(coords):
#     num_nodes = len(coords)
#     distances = [[0] * num_nodes for _ in range(num_nodes)]
#     for i in range(num_nodes):
#         for j in range(num_nodes):
#             if i != j:
#                 lat1, lon1 = coords[i]
#                 lat2, lon2 = coords[j]
#                 distances[i][j] = haversine(lat1, lon1, lat2, lon2)
#     return distances

# def solve_ctop_with_budget_constraint(num_nodes, num_vehicles, distances, scores, demands, vehicle_capacity, travel_cost_per_unit, budget):
#     solver = pywraplp.Solver.CreateSolver('CBC')

#     # 변수 정의
#     x = {}
#     y = {}
#     for i in range(num_nodes):
#         for j in range(num_nodes):
#             for k in range(num_vehicles):
#                 x[i, j, k] = solver.BoolVar(f'x_{i}_{j}_{k}')
#         for k in range(num_vehicles):
#             y[i, k] = solver.BoolVar(f'y_{i}_{k}')

#     # 목적 함수 정의 (총 점수 최대화)
#     solver.Maximize(
#         solver.Sum(scores[i] * y[i, k] for i in range(1, num_nodes) for k in range(num_vehicles))
#     )

#     # 제약 조건 정의
#     # 1. 각 노드는 하나의 차량에만 할당될 수 있음
#     for i in range(1, num_nodes):
#         solver.Add(solver.Sum(y[i, k] for k in range(num_vehicles)) <= 1)

#     # 2. 차량별 제약 조건
#     for k in range(num_vehicles):
#         # 차량 용량 제약
#         solver.Add(solver.Sum(demands[i] * y[i, k] for i in range(1, num_nodes)) <= vehicle_capacity[k])

#         # 차량의 경로 흐름 제약
#         for i in range(num_nodes):
#             solver.Add(solver.Sum(x[i, j, k] for j in range(num_nodes) if j != i) == y[i, k])
#             solver.Add(solver.Sum(x[j, i, k] for j in range(num_nodes) if j != i) == y[i, k])

#         # 차량은 Depot(출발지)에서 출발하고 돌아와야 함
#         solver.Add(solver.Sum(x[0, j, k] for j in range(1, num_nodes)) == 1)
#         solver.Add(solver.Sum(x[i, 0, k] for i in range(1, num_nodes)) == 1)

#     # 총 이동 비용 제약 (강화된 예산 제약)
#     solver.Add(
#         solver.Sum(
#             distances[i][j] * travel_cost_per_unit * x[i, j, k]
#             for i in range(num_nodes) for j in range(num_nodes) for k in range(num_vehicles)
#         ) <= budget)

#     # 문제 풀이
#     status = solver.Solve()

#     if status == pywraplp.Solver.OPTIMAL:
#         total_score, total_cost, total_demand_covered, total_distance = 0, 0, 0, 0
#         routes = []
#         for k in range(num_vehicles):
#             route = [0]
#             for i in range(1, num_nodes):
#                 if y[i, k].solution_value() > 0.5:
#                     route.append(i)
#             route.append(0)
#             routes.append(route)

#             travel_cost = sum(distances[route[i]][route[i + 1]] * travel_cost_per_unit for i in range(len(route) - 1))
#             total_distance += sum(distances[route[i]][route[i + 1]] for i in range(len(route) - 1))
#             total_cost += travel_cost
#             total_demand_covered += sum(demands[i] for i in route if i != 0)
#             total_score += sum(scores[i] for i in route if i != 0)

#         return total_score, total_cost, total_demand_covered, total_distance, routes
#     return None, None, None, None, None

# def visualize_routes_on_map(coords, routes, num_vehicles):
#     # 중앙 위치를 울산으로 설정
#     m = folium.Map(location=[35.5299, 129.3299], zoom_start=13)

#     # 차량 경로 색상을 설정
#     colors = ['red', 'green', 'blue', 'purple', 'orange']
    
#     # 경로를 지도에 추가
#     for k, route in enumerate(routes):
#         color = colors[k % len(colors)]
        
#         # 경로의 각 지점을 PolyLine으로 연결하여 지도에 추가
#         folium.PolyLine(
#             locations=[(coords[node][1], coords[node][0]) for node in route],
#             color=color,
#             weight=5,
#             opacity=0.7,
#             popup=f'Vehicle {k + 1} Route'
#         ).add_to(m)
        
#         # 각 노드에 마커 추가
#         for node in route:
#             folium.Marker(
#                 location=[coords[node][1], coords[node][0]],
#                 popup=f'Node {node}',
#                 icon=folium.Icon(color='blue' if node == 0 else 'green')
#             ).add_to(m)
    
#     # 지도를 출력
#     return m

# def solve_ctop_with_budget_range(num_nodes, num_vehicles, distances, scores, demands, vehicle_capacity, travel_cost_per_unit, budget_range):
#     best_solution = None
#     best_total_cost = float('inf')
    
#     # 예산 범위에 대해 모델을 여러 번 실행
#     for budget in budget_range:
#         total_score, total_cost, total_demand_covered, total_distance, routes = solve_ctop_with_budget_constraint(
#             num_nodes, num_vehicles, distances, scores, demands, vehicle_capacity, travel_cost_per_unit, budget
#         )
        
#         # 비용이 더 적은 경우, 최적의 결과로 갱신
#         if total_cost is not None and total_cost < best_total_cost:
#             best_solution = (total_score, total_cost, total_demand_covered, total_distance, routes)
#             best_total_cost = total_cost
    
#     return best_solution


# # GeoDataFrame 데이터 입력
# depot_point = loads(final_gdf.iloc[0]['facility_g'])  # 문자열을 Point 객체로 변환
# depot_coord = (depot_point.x, depot_point.y)  # Point 객체에서 x, y 좌표 추출

# node_coords = [(point.x, point.y) for point in final_gdf.geometry]
# coords = [depot_coord] + node_coords

# scores = [0] + final_gdf['priority_score'].tolist()
# demands = [0] + final_gdf['demand'].tolist()

# distances = calculate_distances(coords)
# num_nodes = len(coords)
# num_vehicles = 5
# vehicle_capacity = [45] * num_vehicles
# travel_cost_per_unit = 6

# # 예산 범위 설정 (예: 50에서 100까지 10씩 증가)
# budget_range = range(40, 150, 5)

# # 문제 풀이
# best_solution = solve_ctop_with_budget_range(
#     num_nodes, num_vehicles, distances, scores, demands, vehicle_capacity, travel_cost_per_unit, budget_range
# )

# # 최적의 결과가 존재할 경우 출력
# if best_solution:
#     total_score, total_cost, total_demand_covered, total_distance, routes = best_solution

#     # 전체 수요량 계산
#     total_demand = sum(demands[1:])  # depot을 제외한 수요량의 합

#     # 테이블 데이터 생성
#     route_data = []
#     for k, route in enumerate(routes):
#         route_distance = sum(distances[route[i]][route[i + 1]] for i in range(len(route) - 1))
#         route_cost = sum(distances[route[i]][route[i + 1]] * travel_cost_per_unit for i in range(len(route) - 1))
#         route_score = sum(scores[node] for node in route if node != 0)
#         route_demand_covered = sum(demands[node] for node in route if node != 0)
        
#         # 비율 계산
#         coverage_percentage = (route_demand_covered / total_demand) * 100

#         route_data.append({
#             "Vehicle": f"Vehicle {k + 1}",
#             "Route": " -> ".join(map(str, route)),
#             "Distance (km)": round(route_distance, 2),
#             "Cost ($)": round(route_cost, 2),
#             "Score": round(route_score, 2),
#             "Demand Covered": route_demand_covered,
#             "Coverage (%)": round(coverage_percentage, 2)  # 비율 추가
#         })

#     # 성과 요약 추가
#     total_summary = {
#         "Vehicle": "Total",
#         "Route": "-",
#         "Distance (km)": round(total_distance, 2),
#         "Cost ($)": round(total_cost, 2),
#         "Score": round(total_score, 2),
#         "Demand Covered": total_demand_covered,
#         "Coverage (%)": round((total_demand_covered / total_demand) * 100, 2)  # 전체 비율 계산
#     }
#     route_data.append(total_summary)

#     # 데이터프레임으로 변환 후 출력
#     df = pd.DataFrame(route_data)

#     # final_gdf['facility_i']의 첫 번째 값을 모든 행에 넣기
#     facility_value = final_gdf['facility_i'].iloc[0]
#     df.insert(0, 'Facility', [facility_value] * len(df))

#     # pandas의 출력 옵션 설정: 소수점 자릿수 제한
#     pd.set_option('display.float_format', '{:.2f}'.format)

#     display(df.style.hide(axis="index"))

#     m = visualize_routes_on_map(coords, routes, num_vehicles)
#     display(m)

# else:
#     print("No solution found.")